##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Jogando o desafio CartPole com o método Actor-Critic


<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/tutorials/reinforcement_learning/actor_critic">     <img src="https://www.tensorflow.org/images/tf_logo_32px.png">     Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/tutorials/reinforcement_learning/actor_critic.ipynb">     <img src="https://www.tensorflow.org/images/colab_logo_32px.png">     Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/tutorials/reinforcement_learning/actor_critic.ipynb">     <img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">     Ver fonte no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/tutorials/reinforcement_learning/actor_critic.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

Este tutorial demonstra como implementar o método [Actor-Critic](https://papers.nips.cc/paper/1786-actor-critic-algorithms.pdf) usando o TensorFlow para treinar um agente no ambiente [`CartPole-v0`](https://www.gymlibrary.dev/environments/classic_control/cart_pole/) do [Open AI Gym](https://www.gymlibrary.dev/). Presume-se que o leitor tenha alguma familiaridade com os [métodos de gradiente de política](https://papers.nips.cc/paper/1713-policy-gradient-methods-for-reinforcement-learning-with-function-approximation.pdf) do [ aprendizado (profundo) por reforço](https://en.wikipedia.org/wiki/Deep_reinforcement_learning).


**Métodos Actor-Critic**

Os métodos Actor-Critic são métodos de [aprendizado de diferença temporal (TD)](https://en.wikipedia.org/wiki/Temporal_difference_learning) que representam a "função política" independente da função de valor.

Uma política ou função política retorna uma distribuição de probabilidade das ações que o agente pode realizar com base no estado específico. Uma função de valor determina o retorno esperado para um agente que começa em um determinado estado e age sempre de acordo com uma certa política depois disso.

No método Actor-Critic, a política refere-se ao *ator* que propõe um conjunto de ações possíveis considerando um estado, e a função de valor estimado refere-se ao *crítico*, que avalia as ações tomadas pelo *ator* com base na política específica.

Neste tutorial, ambos o *Actor* e o *Critic* serão representados usando uma rede neural com duas saídas.


**`CartPole-v0`**

No [ambiente `CartPole-v0`](https://www.gymlibrary.dev/environments/classic_control/cart_pole/), um pêndulo é conectado a um carrinho que se move por uma trilha sem atrito. O pêndulo começa na posição vertical e o objetivo do agente é evitar que ele caia ao aplicar uma força de `-1` ou `+1` ao carrinho. Uma recompensa de `+1` é concedida sempre que o pêndulo permanece na vertical. Um episódio termina quando: 1) o pêndulo está mais de 15 graus fora da vertical; ou 2) o carrinho se move mais de 2,4 unidades do centro.

<center>
  <pre data-md-type="custom_pre">&lt;figure&gt;
    &lt;image src="https://tensorflow.org/tutorials/reinforcement_learning/images/cartpole-v0.gif"&gt;
    &lt;figcaption&gt;
      Trained actor-critic model in Cartpole-v0 environment
    &lt;/figcaption&gt;
  &lt;/figure&gt;</pre>
</center>


O problema é considerado "solucionado" quando a recompensa média total para o episódio alcança 195 em 100 testes consecutivos.

## Configuração

Importe os pacotes necessários e ajuste as configurações globais.


In [ ]:
!pip install gym[classic_control]
!pip install pyglet

In [ ]:
%%bash
# Install additional packages for visualization
sudo apt-get install -y python-opengl > /dev/null 2>&1
pip install git+https://github.com/tensorflow/docs > /dev/null 2>&1

In [ ]:
import collections
import gym
import numpy as np
import statistics
import tensorflow as tf
import tqdm

from matplotlib import pyplot as plt
from tensorflow.keras import layers
from typing import Any, List, Sequence, Tuple


# Create the environment
env = gym.make("CartPole-v1")

# Set seed for experiment reproducibility
seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

# Small epsilon value for stabilizing division operations
eps = np.finfo(np.float32).eps.item()

## O modelo

O *Actor* e o *Critic* serão modelados usando uma rede neural que gera as probabilidades das ações e o valor do Critic, respectivamente. Este tutorial usa as subclasses de modelo para definir o modelo.

Durante a passagem direta, o modelo usará o estado como entrada e gerará as probabilidades das ações e o valor do crítico $V$, que modela a [função de valor](https://spinningup.openai.com/en/latest/spinningup/rl_intro.html#value-functions) dependente do estado. A meta é treinar um modelo que escolha ações com base em uma política $\pi$ que maximiza o [retorno](https://spinningup.openai.com/en/latest/spinningup/rl_intro.html#reward-and-return) esperado.

Para `CartPole-v0`, há quatro valores que representam o estado: a posição do carrinho, a velocidade do carrinho, o ângulo do pêndulo e a velocidade do pêndulo, respectivamente. O agente pode realizar duas ações para empurrar o carrinho para a esquerda (`0`) e a direita (`1`), respectivamente.

Consulte a [página com a documentação sobre o Cart Pole do Gym](https://www.gymlibrary.dev/environments/classic_control/cart_pole/) e o artigo [*Neuronlike adaptive elements that can solve difficult learning control problems*](http://www.derongliu.org/adp/adp-cdrom/Barto1983.pdf) (Elementos adaptativos semelhantes a neurônios que podem resolver problemas difíceis de controle de aprendizado) de Barto, Sutton e Anderson (1983) para mais informações.


In [ ]:
class ActorCritic(tf.keras.Model):
  """Combined actor-critic network."""

  def __init__(
      self, 
      num_actions: int, 
      num_hidden_units: int):
    """Initialize."""
    super().__init__()

    self.common = layers.Dense(num_hidden_units, activation="relu")
    self.actor = layers.Dense(num_actions)
    self.critic = layers.Dense(1)

  def call(self, inputs: tf.Tensor) -> Tuple[tf.Tensor, tf.Tensor]:
    x = self.common(inputs)
    return self.actor(x), self.critic(x)

In [ ]:
num_actions = env.action_space.n  # 2
num_hidden_units = 128

model = ActorCritic(num_actions, num_hidden_units)

## Treine o agente

Para treinar o agente, siga estas etapas:

1. Execute o agente no ambiente para coletar dados de treinamento por episódio.
2. Compute o retorno esperado em cada timestep.
3. Compute a perda para o modelo Actor-Critic combinado.
4. Compute os gradientes e atualize os parâmetros da rede.
5. Repita as etapas 1 a 4 até alcançar os critérios de sucesso ou o máximo de episódios.


### 1. Colete os dados de treinamento

No aprendizado supervisionado, para treinar o modelo Actor-Critic, você precisa ter dados de treinamento. No entanto, para coletar esses dados, o modelo precisaria ser "executado" no ambiente.

Os dados de treinamento são coletados para cada episódio. Depois, em cada timestep, a passagem direta do modelo será executada no estado do ambiente para gerar as probabilidades das ações e o valor do crítico com base na política atual parametrizada pelos pesos do modelo.

A próxima ação será tomada das probabilidades das ações geradas pelo modelo, que depois seria aplicada ao ambiente, fazendo com que o próximo estado e recompensa sejam gerados.

Esse processo é implementado na função `run_episode`, que usa as operações do TensorFlow para depois compilar em um grafo do TensorFlow e treinar mais rapidamente. Observe que `tf.TensorArray`s foram usados para apoiar a iteração do Tensor em arrays de comprimentos variados.

In [ ]:
# Wrap Gym's `env.step` call as an operation in a TensorFlow function.
# This would allow it to be included in a callable TensorFlow graph.

def env_step(action: np.ndarray) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
  """Returns state, reward and done flag given an action."""

  state, reward, done, truncated, info = env.step(action)
  return (state.astype(np.float32), 
          np.array(reward, np.int32), 
          np.array(done, np.int32))


def tf_env_step(action: tf.Tensor) -> List[tf.Tensor]:
  return tf.numpy_function(env_step, [action], 
                           [tf.float32, tf.int32, tf.int32])

In [ ]:
def run_episode(
    initial_state: tf.Tensor,  
    model: tf.keras.Model, 
    max_steps: int) -> Tuple[tf.Tensor, tf.Tensor, tf.Tensor]:
  """Runs a single episode to collect training data."""

  action_probs = tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True)
  values = tf.TensorArray(dtype=tf.float32, size=0, dynamic_size=True)
  rewards = tf.TensorArray(dtype=tf.int32, size=0, dynamic_size=True)

  initial_state_shape = initial_state.shape
  state = initial_state

  for t in tf.range(max_steps):
    # Convert state into a batched tensor (batch size = 1)
    state = tf.expand_dims(state, 0)
  
    # Run the model and to get action probabilities and critic value
    action_logits_t, value = model(state)
  
    # Sample next action from the action probability distribution
    action = tf.random.categorical(action_logits_t, 1)[0, 0]
    action_probs_t = tf.nn.softmax(action_logits_t)

    # Store critic values
    values = values.write(t, tf.squeeze(value))

    # Store log probability of the action chosen
    action_probs = action_probs.write(t, action_probs_t[0, action])
  
    # Apply action to the environment to get next state and reward
    state, reward, done = tf_env_step(action)
    state.set_shape(initial_state_shape)
  
    # Store reward
    rewards = rewards.write(t, reward)

    if tf.cast(done, tf.bool):
      break

  action_probs = action_probs.stack()
  values = values.stack()
  rewards = rewards.stack()
  
  return action_probs, values, rewards

### 2. Compute os retornos esperados

A sequência de recompensas para cada timestep $t$, ${r_{t}}^{T}{em0}{t=1}$ coletada durante um episódio é convertida em uma sequência de retornos esperados ${G{/em0}{t}}^{T}_{t=1}$ em que a soma de recompensas é obtida do timestep atual $t$ a $T$ e cada recompensa é multiplicada com um fator de desconto que diminui exponencialmente $\gamma$:

$$G_{t} = \sum^{T}_{t'=t} \gamma^{t'-t}r_{t'}$$

Desde $\gamma\in(0,1)$, as recompensas mais distantes do timestep atual recebem menos peso.

Intuitivamente, o retorno esperado simplesmente sugere que as recompensas agora são melhores do que as recompensas depois. Em um sentido matemático, é para garantir a convergência da soma de recompensas.

Para estabilizar o treinamento, a sequência resultante de retornos também é padronizada (ou seja, não ter desvio padrão de unidade e média).


In [ ]:
def get_expected_return(
    rewards: tf.Tensor, 
    gamma: float, 
    standardize: bool = True) -> tf.Tensor:
  """Compute expected returns per timestep."""

  n = tf.shape(rewards)[0]
  returns = tf.TensorArray(dtype=tf.float32, size=n)

  # Start from the end of `rewards` and accumulate reward sums
  # into the `returns` array
  rewards = tf.cast(rewards[::-1], dtype=tf.float32)
  discounted_sum = tf.constant(0.0)
  discounted_sum_shape = discounted_sum.shape
  for i in tf.range(n):
    reward = rewards[i]
    discounted_sum = reward + gamma * discounted_sum
    discounted_sum.set_shape(discounted_sum_shape)
    returns = returns.write(i, discounted_sum)
  returns = returns.stack()[::-1]

  if standardize:
    returns = ((returns - tf.math.reduce_mean(returns)) / 
               (tf.math.reduce_std(returns) + eps))

  return returns

### 3. A perda Actor-Critic

Como você está usando um modelo Actor-Critic híbrido, a função de perda escolhida é uma combinação de perdas de Actor e Critic para treinamento, conforme mostrado abaixo:

$$L = L_{actor} + L_{critic}$$

#### A perda Actor

A perda Actor é baseada em [gradientes de política com o Critic como linha de base dependente do estado](https://www.youtube.com/watch?v=EKqxumCuAAY&t=62m23s) e computada com estimativas de amostra única (por episódio).

$$L_{actor} = -\sum^{T}_{t=1} \log\pi_{\theta}(a_{t} | s_{t})[G(s_{t}, a_{t})  - V^{\pi}_{\theta}(s_{t})]$$

em que:

- $T$: é o número de timesteps por episódio, que pode variar de acordo com o episódio
- $s_{t}$: é o estado do timestep $t$
- $a_{t}$: é a ação escolhida no timestep $t$ considerando o estado $s$
- $\pi_{\theta}$: é a política (Actor) parametrizada por $\theta$
- $V^{\pi}_{\theta}$: é a função de valor (Critic) também parametrizada por $\theta$
- $G = G_{t}$: é o retorno esperado para um determinado par de ação e estado no timestep $t$

Um termo negativo é adicionado à soma, já que a ideia é maximizar as probabilidades das ações gerando maiores recompensas ao minimizar a perda combinada.

<br>

##### A vantagem

O termo $G - V$ na nossa formulação $L_{actor}$ é chamado de [Vantagem](https://spinningup.openai.com/en/latest/spinningup/rl_intro.html#advantage-functions), que indica o quão melhor é uma ação considerando um estado específico em comparação com uma ação aleatória selecionada de acordo com a política $\pi$ para esse estado.

Embora seja possível excluir uma linha de base, isso pode resultar em alta variância durante o treinamento. E o bom de escolher o crítico $V$ como linha de base é que ele é treinado para ficar o mais próximo possível de $G$, levando a menor variância .

Além disso, sem o Critic, o algoritmo tenta aumentar as probabilidades de ações tomadas em um estado específico com base no retorno esperado, o que talvez não faça muita diferença se as probabilidades relativas entre as ações permanecerem as mesmas.

Por exemplo, imagine que duas ações para um determinado estado geram o mesmo retorno esperado. Sem o Critic, o algoritmo tentaria aumentar a probabilidade dessas ações com base no objetivo $J$. Com o Critic, pode não haver Vantagem ($G - V = 0$). Portanto, nenhum benefício seria obtido ao aumentar as probabilidades das ações, e o algoritmo definiria os gradientes como zero.

<br>

#### A perda Critic

O treinamento de $V$ o mais próximo possível de $G$ pode ser configurado como um problema de regressão com a seguinte função de perda:

$$L_{critic} = L_{\delta}(G, V^{\pi}_{\theta})$$

em que $L_{\delta}$ é a [perda de Huber](https://en.wikipedia.org/wiki/Huber_loss), que é menos sensível a outliers nos dados do que a perda de erro quadrático.


In [ ]:
huber_loss = tf.keras.losses.Huber(reduction=tf.keras.losses.Reduction.SUM)

def compute_loss(
    action_probs: tf.Tensor,  
    values: tf.Tensor,  
    returns: tf.Tensor) -> tf.Tensor:
  """Computes the combined Actor-Critic loss."""

  advantage = returns - values

  action_log_probs = tf.math.log(action_probs)
  actor_loss = -tf.math.reduce_sum(action_log_probs * advantage)

  critic_loss = huber_loss(values, returns)

  return actor_loss + critic_loss

### 4. Defina o passo de treinamento para atualizar os parâmetros

Todos os passos acimas são combinados em um passo de treinamento executado a cada episódio. Todos os passos que levam à função de perda são executados com o contexto `tf.GradientTape` para ativar a diferenciação automática.

Este tutorial usa o otimizador Adam para aplicar os gradientes aos parâmetros do modelo.

A soma das recompensas não descontadas, `episode_reward`, também é computada nesse passo. Esse valor será usado mais tarde para avaliar se os critérios de sucesso foram atendidos.

O contexto `tf.function` é aplicado à função `train_step` para ser compilado em um grafo do TensorFlow chamável, o que pode levar a um treinamento 10x mais veloz.


In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)


@tf.function
def train_step(
    initial_state: tf.Tensor, 
    model: tf.keras.Model, 
    optimizer: tf.keras.optimizers.Optimizer, 
    gamma: float, 
    max_steps_per_episode: int) -> tf.Tensor:
  """Runs a model training step."""

  with tf.GradientTape() as tape:

    # Run the model for one episode to collect training data
    action_probs, values, rewards = run_episode(
        initial_state, model, max_steps_per_episode) 

    # Calculate the expected returns
    returns = get_expected_return(rewards, gamma)

    # Convert training data to appropriate TF tensor shapes
    action_probs, values, returns = [
        tf.expand_dims(x, 1) for x in [action_probs, values, returns]] 

    # Calculate the loss values to update our network
    loss = compute_loss(action_probs, values, returns)

  # Compute the gradients from the loss
  grads = tape.gradient(loss, model.trainable_variables)

  # Apply the gradients to the model's parameters
  optimizer.apply_gradients(zip(grads, model.trainable_variables))

  episode_reward = tf.math.reduce_sum(rewards)

  return episode_reward

### 5. Execute o loop de treinamento

O treinamento é executado ao realizar o passo de treinamento até alcançar os critérios de sucesso ou o número máximo de episódios.

Um registro contínuo de recompensas dos episódios é mantido em uma fila. Depois de atingir 100 testes, a recompensa mais antiga é removida da ponta esquerda (cauda) da fila e a mais nova é adicionada à frente (direita). Uma soma contínua de recompensas também é mantida para a eficiência computacional.

Dependendo do seu runtime, o treinamento pode acabar em menos de um minuto.

In [ ]:
%%time

min_episodes_criterion = 100
max_episodes = 10000
max_steps_per_episode = 500

# `CartPole-v1` is considered solved if average reward is >= 475 over 500 
# consecutive trials
reward_threshold = 475
running_reward = 0

# The discount factor for future rewards
gamma = 0.99

# Keep the last episodes reward
episodes_reward: collections.deque = collections.deque(maxlen=min_episodes_criterion)

t = tqdm.trange(max_episodes)
for i in t:
    initial_state, info = env.reset()
    initial_state = tf.constant(initial_state, dtype=tf.float32)
    episode_reward = int(train_step(
        initial_state, model, optimizer, gamma, max_steps_per_episode))
    
    episodes_reward.append(episode_reward)
    running_reward = statistics.mean(episodes_reward)
  

    t.set_postfix(
        episode_reward=episode_reward, running_reward=running_reward)
  
    # Show the average episode reward every 10 episodes
    if i % 10 == 0:
      pass # print(f'Episode {i}: average reward: {avg_reward}')
  
    if running_reward > reward_threshold and i >= min_episodes_criterion:  
        break

print(f'\nSolved at episode {i}: average reward: {running_reward:.2f}!')

## Visualização

Após o treinamento, é bom visualizar o desempenho do modelo no ambiente. Você pode executar as células abaixo para gerar uma animação GIF da execução de um episódio do modelo. Observe que pacotes adicionais precisam ser instalados para o Gym renderizar as imagens do ambiente corretamente no Colab.

In [ ]:
# Render an episode and save as a GIF file

from IPython import display as ipythondisplay
from PIL import Image

render_env = gym.make("CartPole-v1", render_mode='rgb_array')

def render_episode(env: gym.Env, model: tf.keras.Model, max_steps: int): 
  state, info = env.reset()
  state = tf.constant(state, dtype=tf.float32)
  screen = env.render()
  images = [Image.fromarray(screen)]
 
  for i in range(1, max_steps + 1):
    state = tf.expand_dims(state, 0)
    action_probs, _ = model(state)
    action = np.argmax(np.squeeze(action_probs))

    state, reward, done, truncated, info = env.step(action)
    state = tf.constant(state, dtype=tf.float32)

    # Render screen every 10 steps
    if i % 10 == 0:
      screen = env.render()
      images.append(Image.fromarray(screen))
  
    if done:
      break
  
  return images


# Save GIF image
images = render_episode(render_env, model, max_steps_per_episode)
image_file = 'cartpole-v1.gif'
# loop=0: loop forever, duration=1: play each frame for 1ms
images[0].save(
    image_file, save_all=True, append_images=images[1:], loop=0, duration=1)

In [ ]:
import tensorflow_docs.vis.embed as embed
embed.embed_file(image_file)

## Próximos passos

Este tutorial demonstrou como implementar o método Actor-Critic usando o TensorFlow.

Como próximo passo, você pode tentar treinar um modelo em um ambiente diferente no Gym.

Para mais informações sobre os métodos Actor-Critic e o problema Cartpole-v0, consulte estes materiais:

- [O método Actor-Critic](https://hal.inria.fr/hal-00840470/document)
- [A palestra do Actor-Critic (CAL)](https://www.youtube.com/watch?v=EKqxumCuAAY&list=PLkFD6_40KJIwhWJpGazJ9VSj9CFMkb79A&index=7&t=0s)
- [Problema de controle de aprendizado Cart Pole [Barto, et al. 1983]](http://www.derongliu.org/adp/adp-cdrom/Barto1983.pdf)

Para mais exemplos de aprendizado por reforço no TensorFlow, confira os seguintes recursos:

- [Exemplos de código de aprendizado por reforço (keras.io)](https://keras.io/examples/rl/)
- [Biblioteca TF-Agents de aprendizado por reforço](https://www.tensorflow.org/agents)
